In [1]:
! pip install openstef==3.4.72 jupyter==1.0

In [2]:
# Import all required packages.
from openstef.data_classes.prediction_job import PredictionJobDataClass
from openstef.pipeline.train_model import train_model_pipeline
from IPython.display import IFrame
import pandas as pd

# Set plotly as the default pandas plotting backend.
pd.options.plotting.backend = 'plotly'

# Check if running in Google Colab.
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [50]:
# defining the prediction job, `model_kwargs` contains hyperparams
pj = dict(id=101,
        model='xgb',
        forecast_type="demand",
        horizon_minutes=120,
        resolution_minutes=60,
        name="xgb_poc_1",
        save_train_forecasts=True,
        ignore_existing_models=True,
        model_kwargs = {
          "learning_rate": 0.01,
          "early_stopping_rounds": 10,
          "n_estimators": 500
        },
        quantiles=[0.1, 0.5, 0.9]
       )

pj=PredictionJobDataClass(**pj)

In [51]:
# Inspect your prediction job here.
display(pj)

PredictionJobDataClass(id=101, model='xgb', model_kwargs={'learning_rate': 0.01, 'early_stopping_rounds': 10, 'n_estimators': 500}, forecast_type='demand', horizon_minutes=120, resolution_minutes=60, lat=52.132633, lon=5.291266, name='xgb_poc_1', electricity_bidding_zone=<BiddingZone.NL: 'NL'>, train_components=None, description=None, quantiles=[0.1, 0.5, 0.9], train_split_func=None, backtest_split_func=None, train_horizons_minutes=None, default_modelspecs=None, save_train_forecasts=True, completeness_threshold=0.5, minimal_table_length=100, flatliner_threshold_minutes=1440, detect_non_zero_flatliner=False, data_balancing_ratio=None, rolling_aggregate_features=[], depends_on=[], sid=None, turbine_type=None, n_turbines=None, hub_height=None, pipelines_to_run=[<PipelineType.TRAIN: 'train'>, <PipelineType.HYPER_PARMATERS: 'hyper_parameters'>, <PipelineType.FORECAST: 'forecast'>], alternative_forecast_model_pid=None, data_prep_class=None)

In [52]:
if IN_COLAB:
    input_data=pd.read_csv("/content/master_data_with_forecasted.csv", index_col=0, parse_dates=True)
else:
    input_data=pd.read_csv("../data/master_data_with_forecasted.csv", index_col=0, parse_dates=True)

In [53]:
# Inspect all column names of the input data
print("columns in csv")
print(input_data.columns)

# dropping columns as we want
input_data = input_data.drop(columns=["date_time_com", "forecasted_load"])
print("remaining columns after dropping")
print(input_data.columns)

columns in csv
Index(['load', 'date_time_com', 'Holiday', 'Holiday_Type', 'temp', 'rhum',
       'prcp', 'wdir', 'wspd', 'pres', 'cldc', 'coco', 'forecasted_load'],
      dtype='object')
remaining columns after dropping
Index(['load', 'Holiday', 'Holiday_Type', 'temp', 'rhum', 'prcp', 'wdir',
       'wspd', 'pres', 'cldc', 'coco'],
      dtype='object')


In [54]:
pd.options.display.max_columns = None
display(input_data.head())

,load,Holiday,Holiday_Type,temp,rhum,prcp,wdir,wspd,pres,cldc,coco
date_time,,,,,,,,,,,
2023-01-01 06:00:00+00:00,834.0,0.0,0.0,22.0,60.0,0.0,340.0,7.6,1020.2,1.0,1.0
2023-01-01 07:00:00+00:00,736.0,0.0,0.0,22.7,53.0,0.0,9.0,1.8,1018.2,1.0,1.0
2023-01-01 08:00:00+00:00,720.0,0.0,0.0,23.4,49.0,0.0,354.0,1.8,1017.3,1.0,1.0
2023-01-01 09:00:00+00:00,690.0,0.0,0.0,23.7,51.0,0.0,0.0,0.0,1017.2,0.0,1.0
2023-01-01 10:00:00+00:00,668.0,0.0,0.0,22.0,59.0,0.0,302.0,1.8,1016.9,0.0,1.0


In [55]:
# Here we are defining the limit of training data
# print(input_data.shape)
print(input_data.index.get_loc('2023-01-01 06:00:00+00:00'))
print(input_data.index.get_loc('2025-06-15 23:00:00+00:00'))
traing_data_last_index = input_data.index.get_loc('2025-06-15 23:00:00+00:00')

train_data=input_data.iloc[:traing_data_last_index+1]

0
21521


In [56]:
# checking if the limit of training data matches our expectation
print(f"starting hour of training_data {train_data.head(1).index}")
print(f"ending hour of training_data {train_data.tail(1).index}")

starting hour of training_data DatetimeIndex(['2023-01-01 06:00:00+00:00'], dtype='datetime64[ns, UTC]', name='date_time', freq=None)
ending hour of training_data DatetimeIndex(['2025-06-15 23:00:00+00:00'], dtype='datetime64[ns, UTC]', name='date_time', freq=None)


In [57]:
# cleaning up training data by removing duplicate indices and non-datetime indices
# Remove duplicate index values from train_data
train_data = train_data[~train_data.index.duplicated(keep='first')]

# Remove rows with NaT in the index
train_data = train_data[train_data.index.notna()]

In [58]:
# traning the model
import os

mlflow_dir = "./mlflow_trained_models"
mlflow_tracking_uri = os.path.abspath(mlflow_dir)

train_data, validation_data, test_data = train_model_pipeline(
    pj,
    train_data,
    check_old_model_age=False,
    mlflow_tracking_uri=mlflow_tracking_uri,
    artifact_folder="./mlflow_artifacts",
)

2025/11/05 03:30:05 WARNING mlflow.tracking.fluent: Cannot retrieve experiment by name 101


2025-11-05 03:30:05 [warning  ] No old model found, training new model pid=101
[0]	validation_0-rmse:293.17984	validation_1-rmse:306.81085
[1]	validation_0-rmse:290.91744	validation_1-rmse:304.94908
[2]	validation_0-rmse:288.67992	validation_1-rmse:303.11588
[3]	validation_0-rmse:286.46641	validation_1-rmse:301.29909
[4]	validation_0-rmse:284.28054	validation_1-rmse:299.51224
[5]	validation_0-rmse:282.11381	validation_1-rmse:297.75046
[6]	validation_0-rmse:279.97071	validation_1-rmse:296.02200
[7]	validation_0-rmse:277.85507	validation_1-rmse:294.32122
[8]	validation_0-rmse:275.75485	validation_1-rmse:292.63315
[9]	validation_0-rmse:273.67475	validation_1-rmse:290.96031
[10]	validation_0-rmse:271.62384	validation_1-rmse:289.33624
[11]	validation_0-rmse:269.58990	validation_1-rmse:287.69559
[12]	validation_0-rmse:267.58472	validation_1-rmse:286.10466
[13]	validation_0-rmse:265.59585	validation_1-rmse:284.52407
[14]	validation_0-rmse:263.63471	validation_1-rmse:282.97014
[15]	validation_

2025/11/05 03:30:46 INFO mlflow.tracking.fluent: Experiment with name '101' does not exist. Creating a new experiment.


2025-11-05 03:30:46 [info     ] No previous model found in MLflow experiment_name=101
2025-11-05 03:30:51 [info     ] Model saved with MLflow        experiment_name=101
2025-11-05 03:30:53 [info     ] Logged figures to MLflow.
2025-11-05 03:30:53 [info     ] Writing reports to ./mlflow_artifacts/101


In [59]:
# checking if the limit of test data matches our expectation
test_data=input_data.iloc[traing_data_last_index+1:traing_data_last_index+25]
# print(test_data.head())

print(f"starting hour of test_data {test_data.head(1).index}")
print(f"ending hour of test_data {test_data.tail(1).index}")

starting hour of test_data DatetimeIndex(['2025-06-16 00:00:00+00:00'], dtype='datetime64[ns, UTC]', name='date_time', freq=None)
ending hour of test_data DatetimeIndex(['2025-06-16 23:00:00+00:00'], dtype='datetime64[ns, UTC]', name='date_time', freq=None)


In [60]:
import numpy as np
from openstef.pipeline.create_forecast import create_forecast_pipeline

# Prepare data to make the forecast.
realised=input_data.loc[test_data.index, 'load'].copy(deep=True)
to_forecast_data=input_data.copy(deep=True)
to_forecast_data.loc[test_data.index, 'load']=np.nan #clear the load data for the part you want to forecast

In [62]:
# Remove duplicate index values from train_data
to_forecast_data = to_forecast_data[~to_forecast_data.index.duplicated(keep='first')]

# Remove rows with NaT in the index
to_forecast_data = to_forecast_data[to_forecast_data.index.notna()]

# Location where the model was stored in the last exercise.
mlflow_tracking_uri="./mlflow_trained_models"

forecast=create_forecast_pipeline(
    pj,
    to_forecast_data,
    mlflow_tracking_uri,
)

2025-11-05 03:33:05 [warning  ] feature_modules not an attribute of the old model, using None  experiment_name=101
2025-11-05 03:33:05 [info     ] Model successfully loaded with MLflow
2025-11-05 03:33:05 [info     ] Found 24 values of constant load (repeated values), converted to NaN value. cleansing_step=repeated_values frac_values=0.001000541960228457 num_values=24 pj_id=101
2025-11-05 03:33:08 [warning  ] Adding NaN column for missing feature: T-15min missing_feature=T-15min
2025-11-05 03:33:08 [warning  ] Adding NaN column for missing feature: T-30min missing_feature=T-30min
2025-11-05 03:33:08 [warning  ] Adding NaN column for missing feature: T-45min missing_feature=T-45min
2025-11-05 03:33:08 [warning  ] Adding NaN column for missing feature: day_ahead_electricity_price missing_feature=day_ahead_electricity_price
2025-11-05 03:33:08 [warning  ] The provided input data (features) contains more values than is to be expected from analysis expected_numbers_timedelayed=23795.0 featu

In [63]:
# Calculate absolute and percentage differences between realised and forecast
comparison_df = pd.DataFrame({
    'realised': realised,
    'forecast': forecast['forecast'].head(24)
})

comparison_df['absolute_difference'] = (comparison_df['forecast'] - comparison_df['realised']).abs()
comparison_df['percentage_difference'] = (comparison_df['absolute_difference'] / comparison_df['realised']) * 100

display(comparison_df)

,realised,forecast,absolute_difference,percentage_difference
date_time,,,,
2025-06-16 00:00:00+00:00,1481.0,1479.440674,1.559326,0.105289
2025-06-16 01:00:00+00:00,1503.0,1333.142456,169.857544,11.301234
2025-06-16 02:00:00+00:00,1446.0,1311.311157,134.688843,9.314581
2025-06-16 03:00:00+00:00,1427.0,1320.421143,106.578857,7.468736
2025-06-16 04:00:00+00:00,1373.0,1322.404663,50.595337,3.685021
2025-06-16 05:00:00+00:00,1398.0,1405.973389,7.973389,0.570343
2025-06-16 06:00:00+00:00,1424.0,1413.387085,10.612915,0.745289
2025-06-16 07:00:00+00:00,1389.0,1377.587646,11.412354,0.821624
2025-06-16 08:00:00+00:00,1315.0,1269.904541,45.095459,3.429312
